<a href="https://colab.research.google.com/github/Bindhya-K/NLP/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [ ]:
!kaggle datasets download -d algord/fake-news



  0% 0.00/1.68M [00:00<?, ?B/s]
100% 1.68M/1.68M [00:00<00:00, 82.3MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  fake-news.zip
  inflating: FakeNewsNet.csv         


In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Kaggle/FakeNewsNet.csv')

In [ ]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:
df.shape

(23196, 5)

In [ ]:
df.isna().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.shape

(22866, 5)

In [ ]:
X=df.drop('real',axis=1)

In [ ]:
Y=df['real']

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense


In [ ]:
#Data Preprocessing
vocab_size = 10000
messages = X.copy()
messages.reset_index(inplace=True)


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review = re.sub('^[a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review=review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

In [ ]:
corpus

["andi burruss explod rape accus 'real housew atlanta' reunion (video)",
 "eople' choic award 2018: best red carpet look",
 "ophia bush send sweet birthday messag 'one tree hill' co-star hilari burton: 'breyton 4eva'",
 'olombian singer maluma spark rumour inappropri relationship aunt',
 'ossip girl 10 year later: upper east sider shock world chang pop cultur forev',
 'wen stefani got dump blake shelton "jealousi drama" (exclusive)',
 'roward counti sheriff fire lie parkland',
 "mber rose shut french montana date rumors, call rapper 'bruvaaa'",
 "indi kale make first post-babi appear disneyland 'wrinkl time' co-star",
 'atharin mcphee butcher toni nominations: “i drinking”',
 '‘wag miami’ star ashley nicol robert philip wheeler marri',
 'el gibson: hollywood pedophil nowher left hide',
 'edium tyler henri address ‘chilling’ messag kristin cavallari’ deceas brother express read',
 'wt season 27 results: week 5 - disney night',
 '5 reason tarek el moussa overcom latest back injuri',
 'av

In [ ]:
#one Hot Encoding
onehot_repres = [one_hot(words,vocab_size)for words in corpus]

In [ ]:
# Padding
sent_length =20
pre_embedding = pad_sequences(onehot_repres,maxlen=sent_length,padding='pre')

In [ ]:
pre_embedding[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2979,
       1043, 9969, 5474, 1341, 8031, 2591, 5133, 5701, 9584], dtype=int32)

In [ ]:
dim=40
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(vocab_size,dim,input_length=sent_length))
# till now we done the word embedding. npw moving to steps of LSTM
model.add(Dropout(0.33))
model.add(LSTM(100))
model.add(Dropout(0.33))
model.add(Dense(1,activation='sigmoid'))# since it is a classification problem
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            400000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456501 (1.74 MB)
Trainable params: 456501 (1.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(pre_embedding)
Y_final = np.array(Y)
X_final.shape,Y_final.shape

((22866, 20), (22866,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test = train_test_split(X_final,Y_final,test_size=0.33,random_state=42)

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
240/240 [==============================] - 13s 40ms/step - loss: 0.4846 - accuracy: 0.7899 - val_loss: 0.3984 - val_accuracy: 0.8333
Epoch 2/10
240/240 [==============================] - 13s 52ms/step - loss: 0.3424 - accuracy: 0.8560 - val_loss: 0.3941 - val_accuracy: 0.8349
Epoch 3/10
240/240 [==============================] - 12s 49ms/step - loss: 0.2833 - accuracy: 0.8824 - val_loss: 0.4381 - val_accuracy: 0.8281
Epoch 4/10
240/240 [==============================] - 10s 40ms/step - loss: 0.2457 - accuracy: 0.9001 - val_loss: 0.4532 - val_accuracy: 0.8222
Epoch 5/10
240/240 [==============================] - 12s 51ms/step - loss: 0.2133 - accuracy: 0.9140 - val_loss: 0.5008 - val_accuracy: 0.8098
Epoch 6/10
240/240 [==============================] - 11s 46ms/step - loss: 0.1850 - accuracy: 0.9270 - val_loss: 0.5730 - val_accuracy: 0.8072
Epoch 7/10
240/240 [==============================] - 10s 43ms/step - loss: 0.1582 - accuracy: 0.9359 - val_loss: 0.6004 - val_accuracy:

In [ ]:
Y_pred = model.predict(X_test)
Y_pred=np.round(Y_pred).astype(int)
#Y_pred_final = np.argmax(Y_pred,axis=1)
print(Y_pred)
#print(Y_pred_final)


236/236 [==============================] - 2s 6ms/step
[[1]
 [1]
 [1]
 ...
 [0]
 [1]
 [1]]


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))

[[ 975  867]
 [ 613 5091]]
0.8038695997879671


In [ ]:
X_test

array([[   0,    0,    0, ..., 8741, 9400, 7468],
       [   0,    0,    0, ..., 8084, 7240, 6925],
       [   0,    0,    0, ...,  958,  259, 7178],
       ...,
       [   0,    0,    0, ..., 4009, 3931, 1979],
       [   0,    0,    0, ...,    0, 7506, 9480],
       [   0,    0,    0, ..., 2794,  627, 5534]], dtype=int32)